# <font color='Blue'>Import all dependencies</font>

In [1]:
import re
import datetime
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# <font color='red'>Function to Scrape Next Pages</font>

In [2]:
property_url = 'https://meqasa.com'
def next_page(page_num, property_url, requests):
    next_url = property_url + '/properties-for-rent-in-ghana?w=' + str(page_num)
    result = requests.get(next_url)
    print(result)
    if result.status_code == 200:
        print('Scraping page number:', page_num)
    else:
        print('Error occured in page load')
        
    return result, requests, next_url

# <font color='brown'>Perform Scraping</font>

In [3]:
d_property_name=[]
d_address=[]
d_description=[]
d_period=[]
d_currency=[]
d_amount=[]
d_bed=[]
d_shower=[]
d_garage=[]
d_area=[]
d_url=[]

# Enter the number of pages you would like to scrape
number_of_pages = 10

print('========================')
print('==== Scrape Tonaton ====')

for i in range(1, number_of_pages+1):
    
    result, requests, next_url = next_page(i, property_url, requests)
    src  = result.content
    soup = BeautifulSoup(src, 'html.parser')
    containers = soup.findAll('div', {'class': 'mqs-prop-dt-wrapper'})
    for container in containers:
        property_name = container.h2.a.text
        address = container.h2.a.text.split('at')[1]
        description = container.findAll('p')[1].text
        c = container.findAll('p')[0].text.split('/')
        price = container.findAll('p')[0].text.split('/')[0]
        def split (toSplit) :
          currency = "$" if ("$" in toSplit) else "GH₵"
          words = toSplit.split(currency)
          words.insert(1, currency)
          if len(words) == 2:
                words = [None, None, None]
          return words
        
        currency = split(price)[1]
        amount = split(price)[2]

        if len(c) == 1:
            period = None
        elif len(c) == 2:
            period = c[1].replace('\n', '')
        elif len(c) == 3:
            period = c[2].replace('\n', '')
        bed = container.ul.find('li',{'class': 'bed'}).text
        shower = container.ul.find('li',{'class': 'shower'}).text
        g = container.ul.find('li',{'class': 'garage'})        
        
        if g:
            g = g.text
        else:
            g = None
         
        a = container.ul.find('li',{'class': 'area'})     
        if a:
            a = a.text
        else:
            a = None
            
        
        area = a
        garage = g
        p_url = container.h2.a['href']
        listing_url = property_url + p_url
        
        
        d_property_name.append(property_name)
        d_address.append(address)
        d_description.append(description)
        d_period.append(period)
        d_currency.append(currency)
        d_amount.append(amount)
        d_bed.append(bed)
        d_shower.append(shower)
        d_garage.append(garage)
        d_area.append(area)
        d_url.append(listing_url)

        df = pd.DataFrame({
            'Property Name': d_property_name,
            'Address': d_address,
            'Description' : d_description,
            'Currency': d_currency,
            'Amount': d_amount,
            'Period' : d_period,
            'Bed' : d_bed,
            'Shower': d_shower,
            'Garage': d_garage,
            'Area' : d_area,
            'Property URL': d_url
            
        })


==== Scrape Tonaton ====
<Response [200]>
Scraping page number: 1
<Response [200]>
Scraping page number: 2
<Response [200]>
Scraping page number: 3
<Response [200]>
Scraping page number: 4
<Response [200]>
Scraping page number: 5
<Response [200]>
Scraping page number: 6
<Response [200]>
Scraping page number: 7
<Response [200]>
Scraping page number: 8
<Response [200]>
Scraping page number: 9
<Response [200]>
Scraping page number: 10


# <font color='teal'>Display results in DataFrame</font>

In [4]:
df

,Property Name,Address,Description,Currency,Amount,Period,Bed,Shower,Garage,Area,Property URL
0,4 bedroom townhouse for rent at Airport Reside...,"Airport Residential Area, Ghana",4 bedroom townhouse for rent in Airport Reside...,GH₵,"28,890",month,4,4,None,None,https://meqasa.com/townhouse-for-rent-at-Airpo...
1,"4 bedroom house for rent at Airport Hills, Acc...","Airport Hills, Accra, Ghana","Price is GH¢9,263 ($1,950) per month­, inclusi...",$,"1,790",month,4,5,None,300 m2,https://meqasa.com/house-for-rent-at-Airport-H...
2,"4 bedroom townhouse for rent at Cantonments, G...","Cantonments, Ghana",A Beautiful and Newly Built 4 Semi-Detached To...,$,"3,500",month,4,4,None,None,https://meqasa.com/townhouse-for-rent-at-Canto...
3,"4 bedroom apartment for rent at Cantonments, G...","Cantonments, Ghana",A true lifestyle experience offering our cheri...,$,"4,000",month,4,3,None,None,https://meqasa.com/apartment-for-rent-at-Canto...
4,2 bedroom furnished apartment for rent at Asha...,"Ashaley Botwe, Lakeside Est",2 bedroom apartment for short stays.Free WiFi ...,$,50,day,2,3,2,"3,434 m2",https://meqasa.com/apartment-for-rent-at-Ashal...
...,...,...,...,...,...,...,...,...,...,...,...
241,3 bedroom furnished apartment for rent at Laba...,"Labadi, Ghana",3 bedroom furnished apartment for rent in laba...,GH₵,"21,870",month,3,3,None,None,https://meqasa.com/apartment-for-rent-at-Labad...
242,1 bedroom furnished apartment for rent at Cant...,"Cantonment, Ghana",Serviced studio apartment for rent in Cantonme...,GH₵,"8,073",month,1,1,None,None,https://meqasa.com/apartment-for-rent-at-Canto...
243,4 bedroom furnished house for rent at East Leg...,"East Legon, Ghana",This furnished 4 bedroom house is located in t...,GH₵,"21,600",month,4,4,2,None,https://meqasa.com/house-for-rent-at-East-Lego...
244,4 bedroom furnished apartment for rent at Laba...,"Labadi, Ghana",4 bedroom furnished penthouse for rent in laba...,GH₵,"27,000",month,4,4,None,None,https://meqasa.com/apartment-for-rent-at-Labad...


# <font color='gold'>Convert DataFrame to CSV</font>

In [5]:
df.to_csv('Meqasa_Scraping.csv')

# <font color='aquablue'>Function to save Data</font>


_A folder called "*data*" should be in directory_

In [6]:
def save_data(df):
    file_time = datetime.datetime.now()
    file_time = file_time.strftime('%Y-%m-%d')
    fine_name = 'meqasa_'+file_time+'.csv'
    
    df.to_csv('data/'+fine_name, index=False)
    
save_data(df)